Q1.1 - Use of two rigid body transformation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#defining vectors
l_wx = np.array([1, -0.1, 0.4]) #in frame B1
l_yz = np.array([0.3, 0.5, -0.2]) #in frame B2


: 